# Reinforcement Learning, 2024-01 
# Tarea 2 - Programación Dinámica

> Daniel Villar González, 201923374.  
> Daniel Alvarez, 201911320.

**Considere la variación del juego escaleras y serpientes mostrada en la figura:**
<center><img src="imagen-resource-h2.PNG" alt="drawing" width="400"/>
</center> 

- **La meta del jugador es ganar la partida llegando a una de las casillas marcadas en azul.**
- **El jugador pierde la partida si cae en una de las casillas marcadas en rojo.**
- **En cada jugada, antes de lanzar el dado, el jugador decide si quiere avanzar o retroceder el número de casillas indicadas por el dado.**
- **El dado está cargado, con probabilidades p =[p1 p2 p3 p4 p5 p6].**
- **En las casillas 1 y 100 la ficha rebota (si se supera el extremo, se avanza en la otra dirección la cantidad restante).**


## **Librerias**

In [2]:
import numpy as np

## **Modelado de ambiente**

**Modele el esquema de juego tal que cumpla con las condiciones establecidas para el agente en el ambiente**

In [ ]:
class Snakeladders:
    def __init__(self, p_dice, blue_coordinates, red_coordinates):
        self.tablero = np.arange(1, 101)
        self.blue_coordinates = blue_coordinates
        self.red_coordinates = red_coordinates
        self.p_dice= p_dice
        self.actual_pos = 1
    
    def launch_dice(self):
        return np.random.choice(np.arange(1, 7), p=self.p_dice)
    
    def bounce(self, new_position):
        if new_position < 1:
            return 1 - new_position
        elif new_position > 100:
            return 200 - new_position
        else:
            return new_position
    
    def move_token(self, dice):
        new_position = self.actual_pos + dice
        new_position = self.bounce(new_position)
        if new_position in self.blue_coordinates:
            print("BLUE BOX!!! WINNER")
            self.posicion_actual = new_position
            return True
        elif new_position in self.red_coordinates:
            print("RED BOX!! LOSSER :(")
            self.posicion_actual = 0
            return False
        else:
            self.actual_pos = new_position
            return True
    
    def play_game(self):
        while self.actual_pos > 0 and self.actual_pos <= 100:
            print("Your box is: ", self.actual_pos)
            decision = input("Do you want to advance (A) or return (R)? ")
            if decision.upper() == 'A':
                dado = self.launch_dice()
                print("Your dice is :", dado)
            elif decision.upper() == 'R':
                dado = -self.launch_dice()
                print("Your dice is:", abs(dado))
            elif decision.upper() == 'C':
                break
            else:
                print("Invalid decision!!")
                continue
            if not self.move_token(dado):
                print("End Game.")
                break


p_dice = [0.1, 0.1, 0.2, 0.2, 0.2, 0.2]

blue_coordinates = [100, 80]
red_coordinates = [21, 37, 45, 65, 89]

game = Snakeladders(p_dice, blue_coordinates, red_coordinates)
game.play_game()


## **Punto 1**

**Modele este problema como un MDP. De detalladamente todos los elementos del MDP: estados, recompensas, acciones y p(s′, r | s, a) ∀s, s′, r, a, y factor de descuento γ.**

Estados (S): Los estados en este problema representarán la posición actual del jugador en el tablero. Cada casilla en el tablero será un estado posible, desde la casilla 1 hasta la casilla 100: S = {1, 2, 3, ..., 99, 100}.

Recompensas (R): Se asigna una recompensa positiva (+1) cuando el jugador llega a una casilla marcada en azul (la meta) y una recompensa negativa (-1) cuando el jugador cae en una casilla marcada en rojo (pierde la partida). En el resto de las casillas la recompensa será 0. 

Acciones (A): En cada estado, el jugador puede tomar una acción que consiste en decidir si avanza o retrocede el número de casillas indicadas por el dado. Dado que el dado está cargado, el jugador también debe considerar la probabilidad de avanzar o retroceder en función de los resultados posibles del dado: A = {Avanzar, Retroceder}

Probabilidad de transición (p(s', r | s, a)): Define la probabilidad de pasar de un estado s a un estado s′ y recibir una recompensa r dado que se toma una acción a. En este caso, la probabilidad de transición depende de la acción tomada (avanzar o retroceder) y del resultado del dado. Más adelante se definirá la matriz de probabilidades de transición.

Factor de descuento (γ): Se utilizará un valor de descuento γ=0.9 para dar más peso a las recompensas inmediatas.

A continuación se definirán las probabilidades de transición para cada posible estado:

In [127]:
p1 = 1
p2 = 2
p3 = 3
p4 = 4
p5 = 5
p6 = 6
p = [p1, p2, p3, p4, p5, p6]
transiciones = {8:26, 21:82, 43:77, 50:91, 54:93, 62:96, 66:87, 80:100, 98:28, 95:24, 92:51, 83:19, 73:1, 69:33, 64:36, 59:17, 55:7, 52:11, 48:9, 46:5, 44:22}
recompensas = {23:-1, 37:-1, 45:-1, 67:-1, 89:-1, 80:1, 100:1}
    
matrizAdelante = []
for i in range(100):
    matrizAdelante.append([0] * 100)

matrizRecompensasAdelante = []
for i in range(100):
    matrizRecompensasAdelante.append([0] * 100)

llaves_transiciones = transiciones.keys()
llaves_recompensas = recompensas.keys()

for i in range(len(matrizAdelante)): 
    for j in range(0, 6):
        destino = i + j + 1 
        if destino >= 100:  
            destino -= 100  
        if destino+1 in llaves_transiciones:
            matrizAdelante[i][transiciones[destino+1]-1] = p[j]  
            if destino+1 in llaves_recompensas:
                matrizRecompensasAdelante[i][transiciones[destino+1]-1] = recompensas[destino+1]
        else:
            matrizAdelante[i][destino] = p[j] 
            if destino+1 in llaves_recompensas:
                matrizRecompensasAdelante[i][destino] = recompensas[destino+1]
        
for i in range(len(matrizAdelante)):
    print(matrizAdelante[i])

[0, 1, 2, 3, 4, 5, 6, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
[0, 0, 1, 2, 3, 4, 5, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 6, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
[0, 0, 0, 1, 2, 3, 4, 0, 6, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 5, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
[0, 0, 0, 0, 1, 2, 3, 0, 5, 6, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 4, 0, 0, 0, 0, 0, 0, 

Como se puede notar, quedaron definidas las probabilidades de transición desde un s a un estado s′ dado que se toma la acción de ir hacia adelante. Las filas indican el estado s y las columnas el estado s'. El valor de la matriz en la posición (s,s') indica la probabilidad de pasar desde el estado s al estado s', las cuales dependen de los valores de **p**. A continuación se muestra la recompensa obtenida al tomar la acción de ir hacia adelante, desde un estado s:

In [128]:
for i in range(len(matrizRecompensasAdelante)):
    print(matrizRecompensasAdelante[i])

[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 


A continuación se definirán las probabilidades de transición para la acción de ir hacia atrás:

In [133]:
matrizAtras = []
for i in range(100):
    matrizAtras.append([0] * 100)

matrizRecompensasAtras= []
for i in range(100):
    matrizRecompensasAtras.append([0] * 100)

for i in range(len(matrizAtras)): 
    for j in range(0, 6):
        destino = i - j 
        if destino <= 0:  
            destino += 100
        if destino in llaves_transiciones:
            matrizAtras[i][transiciones[destino]-1] = p[j]  
            if destino in llaves_recompensas:
                matrizRecompensasAtras[i][transiciones[destino]-1] = recompensas[destino]
        else:
            matrizAtras[i][destino-1] = p[j] 
            if destino in llaves_recompensas:
                matrizRecompensasAtras[i][destino-1] = recompensas[destino]

for i in range(len(matrizAtras)):
    print(matrizAtras[i])

[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 6, 0, 0, 0, 3, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 5, 4, 0, 2, 1]
[1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 4, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 6, 5, 0, 3, 2]
[2, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 5, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 6, 0, 4, 3]
[3, 2, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 6, 0, 0, 0, 0, 

Como se puede notar, quedaron definidas las probabilidades de transición desde un s a un estado s′ dado que se toma la acción de ir hacia atrás.

In [134]:
for i in range(len(matrizRecompensasAtras)):
    print(matrizRecompensasAtras[i])

[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1]
[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1]
[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1]
[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 

Como se puede notar, quedaron definidas las recompensas obtenidas desde un s a un estado s′ dado que se toma la acción de ir hacia atrás.

De esta forma, quedan definidas las probabilidades de transición p(s′, r | s, a) ∀s, s′, r, a.

In [6]:
class Snakeladders:
    def __init__(self, p_dice, blue_coordinates, red_coordinates):
        self.tablero = np.arange(1, 101)
        self.blue_coordinates = blue_coordinates
        self.red_coordinates = red_coordinates
        self.p_dice= p_dice
        self.actual_pos = 1
        self.state_space = {i: 0 for i in range(1, 101)}

        for pos in blue_coordinates:
            self.state_space[pos] = 1
        for pos in red_coordinates:
            self.state_space[pos] = -1

    def launch_dice(self):
        return np.random.choice(np.arange(1, 7), p=self.p_dice)
    
    def bounce(self, new_position):
        if new_position < 1:
            return 1 - new_position
        elif new_position > 100:
            return 200 - new_position
        else:
            return new_position
    
    def move_token(self, dice):
        new_position = self.actual_pos + dice
        new_position = self.bounce(new_position)
        if new_position in self.blue_coordinates:
            print("BLUE BOX!!! WINNER")
            self.posicion_actual = new_position
            return True
        elif new_position in self.red_coordinates:
            print("RED BOX!! LOSSER :(")
            self.posicion_actual = 0
            return False
        else:
            self.actual_pos = new_position
            return True
    
    def play_game(self):
        while self.actual_pos > 0 and self.actual_pos <= 100:
            print("Your box is: ", self.actual_pos)
            decision = input("Do you want to advance (A) or return (R)? ")
            if decision.upper() == 'A':
                dado = self.launch_dice()
                print("Your dice is :", dado)
            elif decision.upper() == 'R':
                dado = -self.launch_dice()
                print("Your dice is:", abs(dado))
            elif decision.upper() == 'C':
                break
            else:
                print("Invalid decision!!")
                continue
            if not self.move_token(dado):
                print("End Game.")
                break


p_dice = [0.1, 0.1, 0.2, 0.2, 0.2, 0.2]

blue_coordinates = [100, 80]
red_coordinates = [21, 37, 45, 65, 89]

game = Snakeladders(p_dice, blue_coordinates, red_coordinates)
game.play_game()


NameError: name 'self' is not defined

## **Punto 2**

**Escriba un módulo de Python para el MDP formulado. Su implementación debe tener métodos para:**

- **Construir el MDP para una localización dada de las casillas azules y rojas, el vector p, y el factor de descuento γ.**
- **Dada una política π (estocástica o determinística), calcular la función de valor de estado usando programación dinámica.**
- **Dada una política π (estocástica o determinística), determinar si esta política es óptima chequeando las ecuaciones de optimalidad de Bellman.**
- **Hallar una política óptima para una instancia del MDP, usando iteración de política.**
- **Hallar una política óptima para una instancia del MDP, usando iteración de valor.**


### **Construir el MDP para una localización dada de las casillas azules y rojas, el vector p, y el factor de descuento γ.**

A continuación se define un código para construir el MDP dadas las casillas azules, las casillas rojas, el vector **p** y el factor de descuento γ.

El input de las casillas azules (casillasAzules) sería una lista con los números de las casillas.
El input de las casillas rojas (casillasRojas) sería una lista con los números de las casillas.
El input del vector **p** es una lista de 6 posiciones con las probabilidades p1, p2, p3, p4, p5 y p6.
El input del factor de descuento γ (factorDescuento) es un valor entre 0 y 1. 

In [168]:
def construirMDP(casillasAzules, casillasRojas, p, factorDescuento):
    if factorDescuento>1 or factorDescuento<0:
        return "El valor de descuento no está entre los límites permitidos"
    matrizAdelante = []
    for i in range(100):
        matrizAdelante.append([0] * 100)
    matrizRecompensasAdelante = []
    for i in range(100):
        matrizRecompensasAdelante.append([0] * 100)
    matrizAtras = []
    for i in range(100):
        matrizAtras.append([0] * 100)
    matrizRecompensasAtras= []
    for i in range(100):
        matrizRecompensasAtras.append([0] * 100)
    recompensas = {}
    for i in range(len(casillasAzules)):
        recompensas[casillasAzules[i]] = 1
    for i in range(len(casillasRojas)):
        recompensas[casillasRojas[i]] = -1
    transiciones = {8:26, 21:82, 43:77, 50:91, 54:93, 62:96, 66:87, 80:100, 98:28, 95:24, 92:51, 83:19, 73:1, 69:33, 64:36, 59:17, 55:7, 52:11, 48:9, 46:5, 44:22}
    llaves_transiciones = transiciones.keys()
    llaves_recompensas = recompensas.keys()
    for i in range(len(matrizAdelante)): 
        for j in range(0, 6):
            destino = i + j + 1 
            if destino >= 100:  
                destino -= 100  
            if destino+1 in llaves_transiciones:
                matrizAdelante[i][transiciones[destino+1]-1] = p[j]  
                if destino+1 in llaves_recompensas:
                    matrizRecompensasAdelante[i][transiciones[destino+1]-1] = recompensas[destino+1]
            else:
                matrizAdelante[i][destino] = p[j] 
                if destino+1 in llaves_recompensas:
                    matrizRecompensasAdelante[i][destino] = recompensas[destino+1]
    for i in range(len(matrizAtras)): 
        for j in range(0, 6):
            destino = i - j 
            if destino <= 0:  
                destino += 100
            if destino in llaves_transiciones:
                matrizAtras[i][transiciones[destino]-1] = p[j]  
                if destino in llaves_recompensas:
                    matrizRecompensasAtras[i][transiciones[destino]-1] = recompensas[destino]
            else:
                matrizAtras[i][destino-1] = p[j] 
                if destino in llaves_recompensas:
                    matrizRecompensasAtras[i][destino-1] = recompensas[destino]
    return matrizAdelante, matrizAtras, matrizRecompensasAdelante, matrizRecompensasAtras

### INPUTS
casillasAzules = [80, 100]    
casillasRojas = [23, 37, 45, 67, 89]
p = [1/6, 1/6, 1/6, 1/6, 1/6, 1/6]
factorDescuento = 0.9
matrizAdelante, matrizAtras, matrizRecompensasAdelante, matrizRecompensasAtras = construirMDP(casillasAzules, casillasRojas, p, factorDescuento)

A continuación se muestran los resultados obtenidos, es decir, (1) la matriz de probabilidades de transición al tomar la acción de ir hacia adelante, (2) la matriz de recompensas obtenidas al ir hacia adelante, (3) la matriz de probabilidades de transición al tomar la acción de ir hacia atrás y (4) la matriz de recompensas obtenidas al ir hacia atrás.

In [169]:
for i in range(len(matrizAdelante)):
    print(matrizAdelante[i])

[0, 0.16666666666666666, 0.16666666666666666, 0.16666666666666666, 0.16666666666666666, 0.16666666666666666, 0.16666666666666666, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
[0, 0, 0.16666666666666666, 0.16666666666666666, 0.16666666666666666, 0.16666666666666666, 0.16666666666666666, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0.16666666666666666, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
[0, 0, 0, 0.16666666666666666, 0.16666666666666666, 0.16666666666666666, 0.16666666666666666, 0, 0.16666666666666666, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0.16666666666666

In [170]:
for i in range(len(matrizRecompensasAdelante)):
    print(matrizRecompensasAdelante[i])

[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 

In [171]:
for i in range(len(matrizAtras)):
    print(matrizAtras[i])

[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0.16666666666666666, 0, 0, 0, 0.16666666666666666, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0.16666666666666666, 0.16666666666666666, 0, 0.16666666666666666, 0.16666666666666666]
[0.16666666666666666, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0.16666666666666666, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0.16666666666666666, 0.16666666666666666, 0, 0.16666666666666666, 0.16666666666666666]
[0.16666666666666666, 0.16666666666666666, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0.16666666666666666, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0

In [172]:
for i in range(len(matrizRecompensasAtras)):
    print(matrizRecompensasAtras[i])

[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1]
[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1]
[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1]
[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 

### **Dada una política π (estocástica o determinística), calcular la función de valor de estado usando programación dinámica.**

In [184]:
politica = []
for i in range(100):
    politica.append([0.5, 0.5])


In [185]:
def calcularFuncionValorEstado(matrizAdelante, matrizRecompensasAdelante, matrizAtras, matrizRecompensasAtras, politica, factorDescuento, epsilon):
    valores = [0] * len(matrizAdelante)
    convergencia = False
    while not convergencia: 
        valores_previos = valores.copy()
        for estado in range(len(matrizAdelante)):
            nuevo_valor = 0
            # Avanzar
            for estado_siguiente in range(len(matrizAdelante[estado])):
                nuevo_valor += politica[estado][0] * matrizAdelante[estado][estado_siguiente] * (matrizRecompensasAdelante[estado][estado_siguiente] + factorDescuento*valores[estado_siguiente])
            # Retroceder
            for estado_siguiente in range(len(matrizAtras[estado])):
                nuevo_valor += politica[estado][1] * matrizAtras[estado][estado_siguiente] * (matrizRecompensasAtras[estado][estado_siguiente] + factorDescuento*valores[estado_siguiente])
            valores[estado] = nuevo_valor
        # Verificar si se ha alcanzado la convergencia
        dif_maxima = max(abs(valores[estado] - valores_previos[estado]) for estado in range(len(matrizAdelante))) 
        if dif_maxima < epsilon:
            convergencia = True
    return valores

epsilon = 0.001
V = calcularFuncionValorEstado(matrizAdelante, matrizRecompensasAdelante, matrizAtras, matrizRecompensasAtras, politica, factorDescuento, epsilon)
print("La política es: " +str(politica))
print("El valor de descuento es: " + str(factorDescuento))
print("Las probabilidades del dado son: " + str(p))
print("Función de Valor de Estado:")
for i in range(len(V)):
    print("Casilla " +str(i+1)+" tiene un valor de "+ str(V[i]))

La política es: [[0.5, 0.5], [0.5, 0.5], [0.5, 0.5], [0.5, 0.5], [0.5, 0.5], [0.5, 0.5], [0.5, 0.5], [0.5, 0.5], [0.5, 0.5], [0.5, 0.5], [0.5, 0.5], [0.5, 0.5], [0.5, 0.5], [0.5, 0.5], [0.5, 0.5], [0.5, 0.5], [0.5, 0.5], [0.5, 0.5], [0.5, 0.5], [0.5, 0.5], [0.5, 0.5], [0.5, 0.5], [0.5, 0.5], [0.5, 0.5], [0.5, 0.5], [0.5, 0.5], [0.5, 0.5], [0.5, 0.5], [0.5, 0.5], [0.5, 0.5], [0.5, 0.5], [0.5, 0.5], [0.5, 0.5], [0.5, 0.5], [0.5, 0.5], [0.5, 0.5], [0.5, 0.5], [0.5, 0.5], [0.5, 0.5], [0.5, 0.5], [0.5, 0.5], [0.5, 0.5], [0.5, 0.5], [0.5, 0.5], [0.5, 0.5], [0.5, 0.5], [0.5, 0.5], [0.5, 0.5], [0.5, 0.5], [0.5, 0.5], [0.5, 0.5], [0.5, 0.5], [0.5, 0.5], [0.5, 0.5], [0.5, 0.5], [0.5, 0.5], [0.5, 0.5], [0.5, 0.5], [0.5, 0.5], [0.5, 0.5], [0.5, 0.5], [0.5, 0.5], [0.5, 0.5], [0.5, 0.5], [0.5, 0.5], [0.5, 0.5], [0.5, 0.5], [0.5, 0.5], [0.5, 0.5], [0.5, 0.5], [0.5, 0.5], [0.5, 0.5], [0.5, 0.5], [0.5, 0.5], [0.5, 0.5], [0.5, 0.5], [0.5, 0.5], [0.5, 0.5], [0.5, 0.5], [0.5, 0.5], [0.5, 0.5], [0.5, 0.5],

## **Punto 3**

**Utilice su implementación de programación dinámica para hallar la función de valor de:**

**a) La política que siempre avanza.**

**b) La política que avanza con probabilidad 0,7.**


## **Punto 4**

**Utilice su implementación de iteración de política para encontrar una política óptima para la configuración mostrada en la figura para tres vectores p generados aleatoriamente. Chequee que la política encontrada en cada caso es efectivamente óptima. Muestre la política óptima en cada caso (acción en cada casilla). Repita el experimento pero ahora utilizando iteración de valor (usando los mismos p). Experimente en ambos casos con diferentes valores de γ.**

## **Conclusiones**